# Demux

Demultiplex samples based on sequence headers from OBITools `ngsfilter` command.

In [ ]:
from Bio import SeqIO
import pandas as pd
from glob import glob
from pathlib import Path
from tqdm import tqdm

In [ ]:
df = pd.read_csv("../data/fastq/INV2022_V09_ngsfilter.txt", sep="\t", names = "sample extract tag fprimer rprimer F @".split())
records = SeqIO.parse("../data/fastq/ngsfilter/fastp_assigned.out", "fastq")

In [ ]:
handles = {k: open(f"../data/fastq/demux/fastp/{k}_fastp.fastq", "w") for k in df["extract"].to_list()}

In [ ]:
get_sample_from_description = lambda seq: str(seq.description.split("sample=")[-1].split(";")[0])

In [ ]:
for ix, rec in enumerate(records):
    if (ix > 0) and (ix % 100000 == 0):
        print(f"Processed {ix} records.", end="\r")
    sample = get_sample_from_description(rec)
    rec.description = ""
    rec.seq = rec.seq.upper()
    SeqIO.write(rec, handles[sample], "fastq")

In [ ]:
!gzip ../data/fastq/demux/fastp/*.fastq

In [ ]:
samples = {i.split("/")[-1].split("_fastp")[0]: str(Path(i).absolute()) for
           i in glob("../data/fastq/demux/fastp/*.fastq.gz")}


In [ ]:
df = pd.DataFrame()

In [ ]:
df["sampleID"] = samples.keys()
df["forwardReads"] = samples.values()

In [ ]:
df.to_csv("../data/ampliseq/samplesheet.tsv", sep="\t", index=False)